# Notebook 06: Summary & Reflection

## Synthesizing Knowledge

Answer these questions in your own words to reinforce learning.

## Question 1: Permutation Importance

In your own words, define permutation importance and one case it can mislead.

*(Write 3-6 lines here)*

## Question 2: Ridge vs Lasso

When do you prefer Ridge vs Lasso and why?

*(Write 3-6 lines here)*

## Question 3: PCA

What PCA told you about redundancy in your features.

*(Write 3-6 lines here)*

## Question 4: SHAP Pitfalls

One pitfall of SHAP and how you would mitigate it.

*(Write 3-6 lines here)*